In [1]:
import os

import pandas as pd

from dialog import DialogTriplet, dialog_from_file
from metrics import ExampleMetric, get_metric_agreement

In [2]:
metadata_filepath = 'conversation-similarity/conved.csv'
dialogs_dir = 'conversation-similarity/dialogs'

In [3]:
metadata = pd.read_csv(metadata_filepath)

In [4]:
def get_dialog_filepath(root_dir, dialog_id):
    return os.path.join(root_dir, '{0}.json'.format(dialog_id))

In [5]:
dialog_triplets = []
for triplet in metadata.to_dict('records'):
    label = triplet['more_similar_conv'] - 1
    confidence_score = triplet['more_similar_conv_confidence']

    anchor_filepath = get_dialog_filepath(dialogs_dir, triplet['anchor_conv'])
    dialog_1_filepath = get_dialog_filepath(dialogs_dir, triplet['conv_1'])
    dialog_2_filepath = get_dialog_filepath(dialogs_dir, triplet['conv_2'])

    dialog_triplets.append(
        DialogTriplet(
            anchor_dialog=dialog_from_file(anchor_filepath),
            dialog_1=dialog_from_file(dialog_1_filepath),
            dialog_2=dialog_from_file(dialog_1_filepath),
            label=label,
            confidence_score=confidence_score,
        ),
    )

In [6]:
get_metric_agreement(dialog_triplets, ExampleMetric(), 0.8)

0.5125